In [1]:
import pickle
import numpy as np
import optuna
import pandas as pd
from implicit.als import AlternatingLeastSquares
from implicit.cpu.bpr import BayesianPersonalizedRanking
from implicit.cpu.lmf import LogisticMatrixFactorization
from implicit.evaluation import ranking_metrics_at_k
from implicit.nearest_neighbours import CosineRecommender
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

# optuna.logging.set_verbosity(optuna.logging.WARNING)
# optuna.logging.set_verbosity(optuna.logging.ERROR)

# import logging
# logging.getLogger("implicit").setLevel(logging.ERROR)

In [2]:
import multiprocessing
num_threads = multiprocessing.cpu_count()
random_state = 42

implicit это библиоткка для быстрой совместной фильтрации Python для неявных наборов данных.  
Этот проект предоставляет быстрые реализации на Python нескольких различных популярных алгоритмов рекомендаций для наборов данных неявной обратной связи:  
* Метод наименьших квадратов с чередованием, описанный в статьях « Совместная фильтрация для наборов данных с неявной обратной связью» и « Применение метода сопряженных градиентов для совместной фильтрации с неявной обратной связью».
* Персонализированный байесовский рейтинг
* Факторизация логистической матрицы
* Модели «элемент-элемент ближайшего соседа», использующие косинус, TFIDF или BM25 в качестве метрики расстояния

Все модели имеют многопоточные процедуры обучения, использующие Cython и OpenMP для параллельной подгонки моделей среди всех доступных ядер ЦП. Кроме того, модели ALS и BPR имеют собственные ядра CUDA, что позволяет подгонять их на совместимых графических процессорах. Эта библиотека также поддерживает использование библиотек приближенных ближайших соседей, таких как Annoy , NMSLIB и Faiss, для ускорения выработки рекомендаций .

In [3]:
# import psutil

# # Получение информации о памяти
# memory_info = psutil.virtual_memory()
# print(memory_info)

In [4]:
import gc
gc.collect()

0

In [5]:
# import os
# import threadpoolctl

# os.environ['MKL_NUM_THREADS'] = '5'
# os.environ['OPENBLAS_NUM_THREADS'] = '5'
# os.environ['OMP_NUM_THREADS'] = '5'

# threadpoolctl.threadpool_limits(5, "blas")

In [6]:
# # # Производим десериализацию и извлекаем из файла формата pkl
with open('data\events_cl.pkl', 'rb') as pkl_file:
    events = pickle.load(pkl_file)

events = events[['timestamp', 'visitorid', 'event', 'itemid']]
events.head()

,timestamp,visitorid,event,itemid
19,2015-06-02 04:57:58.505,158090,addtocart,10572
35,2015-06-02 05:57:24.638,361387,view,43485
110,2015-06-02 05:09:00.333,712443,view,346655
112,2015-06-02 05:12:21.632,599528,addtocart,356475
130,2015-06-02 05:17:56.276,599528,transaction,356475


In [7]:
# events = events.sample(frac=0.5, random_state=random_state)

In [8]:
# Построен валидационный датасет (разбивка по времени)
# На мой взгляд так более правильно (в соответствии с заданием)

event_type = {
            'view':1,
            'addtocart':2,
            'transaction':10,
            }
events['event'] = events['event'].map(event_type)


split_date = '2015-07-29'
train = events[events['timestamp'].dt.strftime('%Y-%m-%d') < split_date]
test = events[events['timestamp'].dt.strftime('%Y-%m-%d') >= split_date]

test, valid = train_test_split(test, test_size=0.5, random_state=random_state)

events = events[['visitorid', 'event', 'itemid']]
train = train[['visitorid', 'event', 'itemid']]
test = test[['visitorid', 'event', 'itemid']]
valid = valid[['visitorid', 'event', 'itemid']]

print(train.shape[0]/events.shape[0])
print(test.shape[0]/events.shape[0])
print(valid.shape[0]/events.shape[0])

0.6978126560159761
0.15108587119321018
0.1511014727908138


In [9]:
assert (train[['visitorid', 'itemid', 'event']] >= 0).all().all(), "Есть отрицательные значения!"

In [10]:
train_pivot_1 = pd.pivot_table(
                        train,
                        index="visitorid",
                        columns="itemid",
                        values="event",
                        )

In [11]:
test_pivot_1 = pd.pivot_table(
                    test,
                    index="visitorid",
                    columns="itemid",
                    values="event"
                    )


valid_pivot_1 = pd.pivot_table(
                    valid,
                    index="visitorid",
                    columns="itemid",
                    values="event"
                    )

In [12]:
shell = pd.pivot_table(
                events,
                index="visitorid",
                columns="itemid",
                values="event",
                aggfunc=lambda x: 0
                )


print(train_pivot_1.shape)
print(test_pivot_1.shape)

(5413, 2825)
(2542, 1853)


In [13]:
train_pivot = shell + train_pivot_1
test_pivot = shell + test_pivot_1
valid_pivot = shell + valid_pivot_1


train_pivot = (train_pivot + 1).fillna(0)
test_pivot = (test_pivot + 1).fillna(0)
valid_pivot = (valid_pivot + 1).fillna(0)

In [14]:
train_pivot_sparse = csr_matrix(train_pivot.values)
test_pivot_sparse = csr_matrix(test_pivot.values)
valid_pivot_sparse = csr_matrix(valid_pivot.values)

In [15]:
models = [
        AlternatingLeastSquares(random_state=random_state,
                                num_threads=num_threads),
        BayesianPersonalizedRanking(random_state=random_state,
                                    num_threads=num_threads),
        LogisticMatrixFactorization(random_state=random_state,
                                    num_threads=num_threads),
        CosineRecommender(num_threads=num_threads),
        ]

result_map_at10 = []
result_ndcg_at_k = []
result_auc = []


for model in models:
    model.fit(train_pivot_sparse)

    temp = ranking_metrics_at_k(model, train_pivot_sparse, test_pivot_sparse, K=3)
    result_map_at10.append(temp['map'])
    result_ndcg_at_k.append(temp['ndcg'])
    result_auc.append(temp['auc'])


models_name = []
for i in range(len(models)):
    models_name.append(str(models[i]))

c:\Users\olgas\anaconda3\envs\virt_cute_ranking\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
c:\Users\olgas\anaconda3\envs\virt_cute_ranking\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 6 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2542 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/2542 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/2542 [00:00<?, ?it/s]

c:\Users\olgas\anaconda3\envs\virt_cute_ranking\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/3032 [00:00<?, ?it/s]

  0%|          | 0/2542 [00:00<?, ?it/s]

In [16]:
df_rez = pd.DataFrame(data=[models_name, result_map_at10, result_ndcg_at_k, result_auc]).T
df_rez.columns = ['model', 'map_at10', 'result_ndcg_at_k', 'result_auc']
df_rez

,model,map_at10,result_ndcg_at_k,result_auc
0,<implicit.cpu.als.AlternatingLeastSquares obje...,0.00707,0.008644,0.503349
1,<implicit.cpu.bpr.BayesianPersonalizedRanking ...,0.000678,0.000919,0.499878
2,<implicit.cpu.lmf.LogisticMatrixFactorization ...,0.019571,0.024356,0.510427
3,<implicit.nearest_neighbours.CosineRecommender...,0.007737,0.008932,0.503993


Параметры LogisticMatrixFactorization:

* factors: Количество скрытых факторов. Количество латентных факторов, которые модель использует для представления пользователей и элементов.
* learning_rate: Скорость обучения. Определяет размер шага, который модель делает при обновлении весов во время обучения.
* regularization: Параметр регуляризации. Контролирует величину регуляризации для предотвращения переобучения.
* iterations: Количество итераций. Количество итераций для алгоритма обучения.

In [21]:
def opt_Ext(trial):
    # задаем пространство поиска гиперпараметров

    factors = trial.suggest_categorical('factors',
                                        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                         17, 20, 25, 30, 50, 75, 100])
    learning_rate = trial.suggest_categorical('learning_rate',
                                              [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1,
                                               0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    regularization = trial.suggest_categorical('regularization',
                                               [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 2, 3,
                                                4, 5, 6, 7, 8, 9, 10])
    iterations = trial.suggest_categorical('iterations',
                                           [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                            17, 20, 25, 30, 50, 75, 100, 200, 500, 1000, 10000])


    # создаем модель
    model = LogisticMatrixFactorization(factors=factors,
                                        learning_rate=learning_rate,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads = num_threads,
                                        random_state=random_state,
                                        )

    model.fit(train_pivot_sparse)
    score = ranking_metrics_at_k(model, train_pivot_sparse, valid_pivot_sparse, K=3)['ndcg']
    return score

In [25]:
# cоздаем объект исследования
stud = optuna.create_study(direction="maximize")


# ищем лучшую комбинацию гиперпараметров
stud.optimize(opt_Ext, n_trials=150)

[I 2024-11-26 21:31:11,715] A new study created in memory with name: no-name-4199b9b1-0363-4c35-a207-49b49194f1e5


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2587 [00:00<?, ?it/s]

[I 2024-11-26 21:31:12,155] Trial 0 finished with value: 0.0 and parameters: {'factors': 8, 'learning_rate': 0.1, 'regularization': 9, 'iterations': 5}. Best is trial 0 with value: 0.0.


  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/2587 [00:00<?, ?it/s]

[I 2024-11-26 21:46:29,056] Trial 148 finished with value: 0.008397483230039294 and parameters: {'factors': 11, 'learning_rate': 8, 'regularization': 1, 'iterations': 200}. Best is trial 20 with value: 0.02408674876150476.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2587 [00:00<?, ?it/s]

[I 2024-11-26 21:46:29,440] Trial 149 finished with value: 0.00035833462100399023 and parameters: {'factors': 30, 'learning_rate': 8, 'regularization': 0.01, 'iterations': 1}. Best is trial 20 with value: 0.02408674876150476.


In [26]:
print(stud.best_params)

{'factors': 75, 'learning_rate': 2, 'regularization': 6, 'iterations': 200}


In [27]:
best_model_implicit = LogisticMatrixFactorization(
                                                factors=stud.best_params['factors'],
                                                learning_rate=stud.best_params['learning_rate'],
                                                regularization=stud.best_params['regularization'],
                                                iterations=stud.best_params['iterations'],
                                                num_threads = num_threads,
                                                random_state=random_state)


best_model_implicit.fit(train_pivot_sparse)


# # Производим сериализацию и записываем результат в файл формата pkl
with open('models/best_model_implicit.pkl', 'wb') as output:
    pickle.dump(best_model_implicit, output)

  0%|          | 0/200 [00:00<?, ?it/s]

In [28]:
temp = ranking_metrics_at_k(best_model_implicit, train_pivot_sparse, test_pivot_sparse, K=3)
temp

  0%|          | 0/2542 [00:00<?, ?it/s]

{'precision': 0.029427220178665267,
 'map': 0.019942739749978157,
 'ndcg': 0.024468867344123933,
 'auc': 0.5109190840171624}

* Значение AUC ~ 0.5 указывает на случайное предсказание.
* Значение AUC 1.0 указывает на идеальное предсказание.

Высокое значение AUC (ближе к 1) означает, что модель лучше отделяет положительные примеры от отрицательных.  
В нашем случае идет простое угадвание.

Получение рекомендаций

In [29]:
user_id = 274515

train_pivot_list = train_pivot.index.tolist()

flag = 0
try:
    # Получение индекса элемента
    index_user = train_pivot_list.index(user_id)
    print(f"Индекс пользователя {user_id} в списке: {index_user}")
    # Получение рекомендаций
    flag = 1

except ValueError:
    print(f"Пользователь {user_id} не найден в списке.")

if flag:
    recomendations_ids, scores = best_model_implicit.recommend(index_user, train_pivot_sparse[index_user], N=10)
    unique_items = np.array(train_pivot.columns)
    recomendations = unique_items[recomendations_ids].tolist()

    print(f'Recomendations for user {user_id}: {recomendations}')
    print(f'Recomendations for user {user_id}: {scores}')

Индекс пользователя 274515 в списке: 1528
Recomendations for user 274515: [461686, 7943, 213834, 312728, 29196, 445351, 37029, 17478, 48030, 409804]
Recomendations for user 274515: [0.63014764 0.43807432 0.4228501  0.38476017 0.29422966 0.23296584
 0.187371   0.17809014 0.174434   0.17265438]


In [30]:
unique_items = np.array(train_pivot.columns)

In [31]:
# # Производим сериализацию и записываем результат в файл формата pkl
with open('data/train_pivot_list_implicit.pkl', 'wb') as output:
    pickle.dump(train_pivot_list, output)

with open('data/train_pivot_sparse_implicit.pkl', 'wb') as output:
    pickle.dump(train_pivot_sparse, output)

with open('data/unique_items.pkl', 'wb') as output:
    pickle.dump(unique_items, output)